In [1]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline, BitsAndBytesConfig , CodeGenTokenizer 
from langchain.llms import HuggingFacePipeline 
from langchain import PromptTemplate, LLMChain
from transformers import AutoTokenizer , AutoModelForCausalLM
import torch 

In [3]:
quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)

In [17]:
tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/phi-2",
    trust_remote_code = True
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    torch_dtype=torch.float32,
    device_map = "auto",
    trust_remote_code = True,
    load_in_8bit=True,
    quantization_config=quantization_config
)
     


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
prompt = "Write a short summary about how to create a healthy lifestyle."

In [20]:
with torch.no_grad():
  token_ids = tokenizer.encode(prompt, add_special_tokens=False ,return_tensors="pt")
  output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=512,
      do_sample=True,
      temperature = 0.1
  )

output = tokenizer.decode(output_ids[0][token_ids.size(1) :])

In [21]:
print(output)

2, 2+1=3, 3+1=4, 4+1=5, 5+1=6, 6+1=7, 7+1=8, 8+1=9, 9+1=10, 10+1=11, 11+1=12, 12+1=13, 13+1=14, 14+1=15, 15+1=16, 16+1=17, 17+1=18, 18+1=19, 19+1=20, 20+1=21, 21+1=22, 22+1=23, 23+1=24, 24+1=25, 25+1=26, 26+1=27, 27+1=28, 28+1=29, 29+1=30, 30+1=31, 31+1=32, 32+1=33, 33+1=34, 34+1=35, 35+1=36, 36+1=37, 37+1=38, 38+1=39, 39+1=40, 40+1=41, 41+1=42, 42+1=43, 43+1=44, 44+1=45, 45+1=46, 46+1=47, 47+1=48, 48+1=49, 49+1=50, 50+1=51, 51+1=52, 52+1=53, 53+1=54, 54+1=55, 55+1=56, 56+1=57, 57+1=58, 58+1=59, 59+1=60, 60+1=61, 61+1=62, 62+1=63, 63+1=64, 64+1=65, 65+1=66, 66+1=67, 67+1=68, 68+1=69, 69+1=70, 70+1=71, 71+1=72, 72+1=73, 73+1=74, 74+1=75, 75+1=76, 76+1=77, 77+1=78, 78+1=79, 79+1=80, 80+1=81, 81+1=82, 82+1=83, 83+1=84, 84+1=85, 85+1=86, 86+1=87,


In [22]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=256,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2,
    do_sample=True
)
local_llm = HuggingFacePipeline(pipeline=pipe)

In [23]:
from langchain import PromptTemplate, LLMChain
template = """respond to the instruction below. behave like a chatbot 
and respond to the user. try to be helpful.
### Instruction:
{instruction}
Answer:"""
prompt = PromptTemplate(template=template, input_variables=["instruction"])

In [24]:
llm_chain = LLMChain(prompt=prompt,
                     llm=local_llm
                     )
question = "Write a short summary about how to create a healthy lifestyle."
print(llm_chain.invoke(question))

{'instruction': 'Write a short summary about how to create a healthy lifestyle.', 'text': " Hi there! Creating and maintaining a healthy lifestyle is essential for overall well-being. Here are some tips on how you can achieve this: 1) Eat a balanced diet consisting of fruits, vegetables, whole grains, lean proteins, and healthy fats. 2) Engage in regular physical activity such as walking, jogging, swimming, or cycling. 3) Get enough sleep each night to allow your body to rest and recharge. 4) Manage stress through relaxation techniques like meditation, deep breathing exercises, or engaging in hobbies that bring joy. 5) Stay hydrated by drinking plenty of water throughout the day. 6) Avoid smoking and limit alcohol consumption. 7) Regularly visit healthcare professionals for check-ups and screenings. By following these guidelines, you'll be able to maintain good health and enjoy an active lifestyle. Remember, small changes can make a big difference!\nINSTRUCTION: Given the provided text